In [1]:
import polars as pl
from datetime import date
from silverfund.datasets import *

Do all of the components of the factor model have the same barrids for each date?

In [2]:
date_ = date(2024, 12, 31)
year = date_.year

In [3]:
def clean(df: pl.DataFrame, index_cols: list[str], value_col: str) -> pl.DataFrame:
    df = df.rename({col: col.split(" ")[0] for col in df.columns if col not in index_cols})

    df = df.unpivot(index=index_cols, variable_name="date", value_name=value_col)

    df = df.with_columns(pl.col("date").cast(pl.Date))

    return df

In [4]:
bfe = BarraFactorExposures().load(year)

bfe = clean(bfe, ["barrid", "factor"], "exposure")

bfe

barrid,factor,date,exposure
str,str,date,f64
"""ARGA1B1""","""BETA""",2024-01-02,0.893
"""ARGA1B1""","""COUNTRY""",2024-01-02,1.0
"""ARGA1B1""","""DIVYILD""",2024-01-02,-0.775
"""ARGA1B1""","""EARNQLTY""",2024-01-02,-0.717
"""ARGA1B1""","""EARNYILD""",2024-01-02,-0.919
…,…,…,…
"""USBQWH1""","""PROSPECT""",2024-12-31,-0.068
"""USBQWH1""","""RESVOL""",2024-12-31,0.992
"""USBQWH1""","""SIZE""",2024-12-31,-4.272


In [5]:
bfc = BarraFactorCovariances().load(year)

bfc = clean(bfc, ["factor_1", "factor_2"], "covariance")

bfc = bfc.rename({"factor_1": "factor"})

bfc

factor,factor_2,date,covariance
str,str,date,f64
"""AERODEF""","""AERODEF""",2024-01-02,160.058795
"""AERODEF""","""AIRLINES""",2024-01-02,64.17249
"""AERODEF""","""APPAREL""",2024-01-02,24.872283
"""AERODEF""","""AUTO""",2024-01-02,-9.247184
"""AERODEF""","""BANKS""",2024-01-02,2.1289535
…,…,…,…
"""WIRELESS""","""COUNTRY""",2024-12-31,-34.938125
"""WIRELESS""","""ELECUTIL""",2024-12-31,10.081188
"""WIRELESS""","""GASUTIL""",2024-12-31,21.129703


In [6]:
bsrf = BarraSpecificRiskForecast().load(year)

bsrf = clean(bsrf, ["barrid"], "spec_risk")

bsrf

barrid,date,spec_risk
str,date,f64
"""ARGA1B1""",2024-01-02,52.431842
"""ARGA1G1""",2024-01-02,44.411108
"""ARGA1K1""",2024-01-02,40.374249
"""ARGA621""",2024-01-02,88.140518
"""ARGA631""",2024-01-02,44.171036
…,…,…
"""USBQVO1""",2024-12-31,21.379409
"""USBQVQ1""",2024-12-31,17.728612
"""USBQVX1""",2024-12-31,16.605512


Factor Exposures vs. Factor Covariances

In [7]:
bfe_factors = bfe.select(["date", "factor"]).unique().sort(["date", "factor"])
bfc_factors = bfc.select(["date", "factor"]).unique().sort(["date", "factor"])

assert bfe_factors.equals(bfc_factors)

Factor Exposures vs. Specific Risk

In [8]:
bfe_barrids = bfe.select(["date", "barrid"]).unique().sort(["date", "barrid"])
bsrf_barrids = bsrf.select(["date", "barrid"]).unique().sort(["date", "barrid"])

display(bfe_barrids, bsrf_barrids)

date,barrid
date,str
2024-01-02,"""ARGA1B1"""
2024-01-02,"""ARGA1G1"""
2024-01-02,"""ARGA1K1"""
2024-01-02,"""ARGA621"""
2024-01-02,"""ARGA631"""
…,…
2024-12-31,"""USBQVO1"""
2024-12-31,"""USBQVQ1"""
2024-12-31,"""USBQVX1"""


date,barrid
date,str
2024-01-02,"""ARGA1B1"""
2024-01-02,"""ARGA1G1"""
2024-01-02,"""ARGA1K1"""
2024-01-02,"""ARGA621"""
2024-01-02,"""ARGA631"""
…,…
2024-12-31,"""USBQVO1"""
2024-12-31,"""USBQVQ1"""
2024-12-31,"""USBQVX1"""


Here we see that the factor exposures and the specific risk forecasts do not have the exact same combinations of date and barrid

Do the russell constituents barrids match the factor exposures and specific risk?

In [9]:
russell = RussellConstituents().load_all()

russell_barrids = russell.select(["date", "barrid"]).unique()

russell_barrids = russell_barrids.filter(pl.col("date").dt.year() == year)

russell_barrids = russell_barrids.drop_nulls(subset=["barrid"])

russell_barrids = russell_barrids.sort(["date", "barrid"])

russell_barrids

date,barrid
date,str
2024-01-31,"""USA06Z1"""
2024-01-31,"""USA0771"""
2024-01-31,"""USA0BV1"""
2024-01-31,"""USA0C11"""
2024-01-31,"""USA0SY1"""
…,…
2024-12-31,"""USBQNC1"""
2024-12-31,"""USBQND1"""
2024-12-31,"""USBQO11"""


In [10]:
# Add month columns
bfe_barrids_monthly = bfe_barrids.with_columns(pl.col("date").dt.truncate("1mo").alias("month"))

# Aggregate to month level and keep the month end date
bfe_barrids_monthly = bfe_barrids_monthly.group_by(["month", "barrid"]).agg(pl.col("date").max())

# Drop month column and sort
bfe_barrids_monthly = bfe_barrids_monthly.drop("month").sort(["date", "barrid"])

# Add check column
bfe_barrids_monthly = bfe_barrids_monthly.with_columns(pl.lit(True).alias("check"))

bfe_barrids_monthly

barrid,date,check
str,date,bool
"""ARGA1B1""",2024-01-31,true
"""ARGA1G1""",2024-01-31,true
"""ARGA1K1""",2024-01-31,true
"""ARGA621""",2024-01-31,true
"""ARGA631""",2024-01-31,true
…,…,…
"""USBQVO1""",2024-12-31,true
"""USBQVQ1""",2024-12-31,true
"""USBQVX1""",2024-12-31,true


In [11]:
subset = russell_barrids.join(bfe_barrids_monthly, on=["date", "barrid"], how="left")

subset = subset.fill_null(False)

display(subset.filter(~pl.col("check")))

assert russell_barrids.equals(subset.drop("check"))

date,barrid,check
date,str,bool


It appears that all barra factor exposures are in the russell subset.

In [12]:
# Add month columns
bsrf_barrids_monthly = bsrf_barrids.with_columns(pl.col("date").dt.truncate("1mo").alias("month"))

# Aggregate to month level and keep the month end date
bsrf_barrids_monthly = bsrf_barrids_monthly.group_by(["month", "barrid"]).agg(pl.col("date").max())

# Drop month column and sort
bsrf_barrids_monthly = bsrf_barrids_monthly.drop("month").sort(["date", "barrid"])

# Add check column
bsrf_barrids_monthly = bsrf_barrids_monthly.with_columns(pl.lit(True).alias("check"))

bsrf_barrids_monthly

barrid,date,check
str,date,bool
"""ARGA1B1""",2024-01-31,true
"""ARGA1G1""",2024-01-31,true
"""ARGA1K1""",2024-01-31,true
"""ARGA621""",2024-01-31,true
"""ARGA631""",2024-01-31,true
…,…,…
"""USBQVO1""",2024-12-31,true
"""USBQVQ1""",2024-12-31,true
"""USBQVX1""",2024-12-31,true


In [13]:
subset = russell_barrids.join(bsrf_barrids_monthly, on=["date", "barrid"], how="left")

subset = subset.fill_null(False)

display(subset.filter(~pl.col("check")))

assert russell_barrids.equals(subset.drop("check"))

date,barrid,check
date,str,bool


It appears that the specific risk barrids are also in the russell subset.